In [24]:
import sys
!{sys.executable} -m pip install fasttext
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install sklearn

In [26]:
import pandas as pd
import numpy as np
import sklearn
import re
import string
import fasttext

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [32]:
train_data = pd.read_csv('train.csv')
train_data = train_data.drop(columns=['id', 'keyword', 'location'])
test_data = pd.read_csv('test.csv')
test_data = test_data.drop(columns=['id', 'keyword', 'location'])

print(train_data.head(5))
print(test_data.head(5))

                                                text  target
0  Our Deeds are the Reason of this #earthquake M...       1
1             Forest fire near La Ronge Sask. Canada       1
2  All residents asked to 'shelter in place' are ...       1
3  13,000 people receive #wildfires evacuation or...       1
4  Just got sent this photo from Ruby #Alaska as ...       1
                                                text
0                 Just happened a terrible car crash
1  Heard about #earthquake is different cities, s...
2  there is a forest fire at spot pond, geese are...
3           Apocalypse lighting. #Spokane #wildfires
4      Typhoon Soudelor kills 28 in China and Taiwan


In [3]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_at(text):
    at = re.compile(r'\@(\w+)')
    return at.sub(r'', text)

def remove_punct(text):
    table = str.maketrans('','',string.punctuation)
    return text.translate(table)

In [43]:
def cleanTweets(data):
    text = data['text']
    text = text.apply(lambda x: remove_URL(x)) \
               .apply(lambda x: remove_html(x)) \
               .apply(lambda x: remove_emoji(x)) \
               .apply(lambda x: remove_at(x)) \
               .apply(lambda x: remove_punct(x))
    return text.to_numpy()

def obtainTargets(data):
    return data['target'].to_numpy()

In [45]:
print(cleanTweets(train_data))
print(obtainTargets(train_data))

['Our Deeds are the Reason of this earthquake May ALLAH Forgive us all'
 'Forest fire near La Ronge Sask Canada'
 'All residents asked to shelter in place are being notified by officers No other evacuation or shelter in place orders are expected'
 ... 'M194 0104 UTC5km S of Volcano Hawaii '
 'Police investigating after an ebike collided with a car in Little Portugal Ebike rider suffered serious nonlife threatening injuries'
 'The Latest More Homes Razed by Northern California Wildfire  ABC News ']
[1 1 1 ... 1 1 1]


In [6]:
train_data_text = '\n'.join(train_data['text'].astype(str).values)
# print(train_data_text)

In [7]:
with open("data.txt", "w") as file:
    file.write(train_data_text)

In [8]:
! /usr/local/Cellar/jupyterlab/3.2.4/libexec/bin/python3.9 -m pip install fasttext
import fasttext
model = fasttext.train_unsupervised('data.txt', model='skipgram')

Read 0M words
Number of words:  3141
Number of labels: 0
Progress: 100.0% words/sec/thread:   96535 lr:  0.000000 avg.loss:  2.534014 ETA:   0h 0m 0s-0.000008 avg.loss:  2.534014 ETA:   0h 0m 0s


In [9]:
# print(model.words)

In [10]:
print(model['lol'].size)

100


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [12]:
corpus = train_data['text'].to_list()
vectorizer = TfidfVectorizer()

In [13]:
X = vectorizer.fit_transform(corpus)

In [14]:
vectorizer.get_feature_names_out()

array(['0011', '001116', '0025', ..., 'ûónegligence', 'ûótech', 'ûówe'],
      dtype=object)

In [15]:
print(X.shape)

(7613, 15586)


In [16]:
from sklearn.linear_model import LogisticRegression
y = train_data['target'].to_numpy()
clf = LogisticRegression(verbose=2).fit(X, y)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        15587     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.27693D+03    |proj g|=  5.35500D+02

At iterate    1    f=  5.19197D+03    |proj g|=  3.33501D+01

At iterate    2    f=  5.00512D+03    |proj g|=  1.18001D+02

At iterate    3    f=  3.99907D+03    |proj g|=  1.11847D+02

At iterate    4    f=  3.70244D+03    |proj g|=  1.40865D+02

At iterate    5    f=  3.55485D+03    |proj g|=  4.40603D+01

At iterate    6    f=  3.53616D+03    |proj g|=  6.26178D+00

At iterate    7    f=  3.52956D+03    |proj g|=  2.95105D+01

At iterate    8    f=  3.52407D+03    |proj g|=  3.51722D+01

At iterate    9    f=  3.51455D+03    |proj g|=  2.25204D+01

At iterate   10    f=  3.50779D+03    |proj g|=  2.21373D+00

At iterate   11    f=  3.50705D+03    |proj g|=  5.71889D+00

At iterate   12    f=  3.50696D+03    |proj g|=  8.30297D+00

At iterate   13    f=  3.5

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
 This problem is unconstrained.


At iterate   14    f=  3.50544D+03    |proj g|=  2.20042D+00

At iterate   15    f=  3.50415D+03    |proj g|=  6.32767D-01

At iterate   16    f=  3.50390D+03    |proj g|=  1.19481D+00

At iterate   17    f=  3.50384D+03    |proj g|=  2.43939D+00

At iterate   18    f=  3.50379D+03    |proj g|=  7.49720D-01

At iterate   19    f=  3.50377D+03    |proj g|=  4.61691D-01

At iterate   20    f=  3.50374D+03    |proj g|=  7.16354D-02

At iterate   21    f=  3.50373D+03    |proj g|=  3.13213D-01

At iterate   22    f=  3.50372D+03    |proj g|=  3.46485D-01

At iterate   23    f=  3.50372D+03    |proj g|=  5.20671D-01

At iterate   24    f=  3.50372D+03    |proj g|=  1.79517D-01

At iterate   25    f=  3.50372D+03    |proj g|=  3.95216D-02

At iterate   26    f=  3.50372D+03    |proj g|=  1.17087D-01

At iterate   27    f=  3.50372D+03    |proj g|=  2.12786D-01

At iterate   28    f=  3.50372D+03    |proj g|=  2.34554D-02

At iterate   29    f=  3.50372D+03    |proj g|=  1.97889D-01

At itera

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s finished


In [17]:
clf.score(X, y)

0.8823065808485485

In [18]:
from sklearn.metrics import confusion_matrix
confusion_matrix(clf.predict(X), y).ravel()

array([4157,  711,  185, 2560])